In [126]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import json

In [61]:
# Function to remove tags
def remove_tags(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)


In [ ]:
for i in range(41):
    if (i % 10) == 0:
        print(i)

    url = f"https://api.digitale-sammlungen.de/ocr/bsb10589853/{i}"
    ocr_html = requests.get(url).text
    ocr_converted = ocr_html.encode('iso-8859-1').decode('utf-8')

    ocr_text = remove_tags(ocr_converted)
    text = re.sub('bsb10589853_[0-9]{5} ', '', ocr_text)
    text = re.sub(r' [\/,.:!;=\(\)\-](?= )', '', text)
    text = re.sub(r'ſ', 's', text)
    #text = re.sub(r' [0-9]+.?(?= )', '', text)
    text = re.sub(r'\s+', ' ', text)

    with open(f"source_texts/praxis_pietatis_melica/{i}.txt", "w") as f:
        f.write(text)

In [113]:
def remove_accents(text):
    # Remove accents except for umlaut characters
    return ''.join(unicodedata.normalize('NFKD', c) if c not in ['ä', 'ö', 'ü', 'Ä', 'Ö', 'Ü'] else c for c in text )

In [164]:
def remove_hyphens(verse):
    return verse

In [163]:
def clean_verse(verse, common_ocr_mistakes):
    verse = verse.lower()
    verse = remove_accents(verse)
    verse = re.sub(r'\s+', ' ', verse)
    verse = re.sub(r' ?[-=] ?', '', verse)
    verse = re.sub(r' ?ij\.? ?', ' ', verse)
    verse = re.sub(r'ſ', 's', verse)
    verse = re.sub(r'&', 'et', verse)
    verse = re.sub(r'[,.:!;\(\)_]', '', verse)
    verse = re.sub(r' ?[0-9]+\.? ?', '', verse)
    verse = re.sub(r'[/,;:]', ' ', verse)
    verse = re.sub(r'[^[A-Za-z0-9äöüßÄÖÜæœ\s]','',verse)

    all_words = []

    for word in verse.split():
        if word in common_ocr_mistakes:
            all_words.append(common_ocr_mistakes[word])
        else:
            all_words.append(word)

    return " ".join(all_words).strip()

In [165]:
with open("common_ocr_mistakes.json", "r") as json_file:
    common_ocr_mistakes = json.load(json_file)

In [166]:
for i in range(41,1291):
    url = f"https://api.digitale-sammlungen.de/ocr/bsb10589853/{i}"
    ocr_html = requests.get(url).text
    ocr_converted = ocr_html.encode('iso-8859-1').decode('utf-8')
    ocr_soup = BeautifulSoup(ocr_converted, "html.parser")
    stanzas = ocr_soup.find_all('p', class_="ocr_par")

    all_verses = []

    for stanza in stanzas:
        verses = re.split('/|,|\.', stanza.text)
        word_collector = ""
        for verse in verses:
            verse = clean_verse(verse, common_ocr_mistakes)
            word_collector += " " + verse
            if len(word_collector.split()) >= 4:
                all_verses.append(word_collector.strip())
                word_collector = ""

    with open(f"source_texts/praxis_pietatis_verses/{i}.json", "w") as f:
        json.dump(all_verses, f, indent=2, ensure_ascii=False)



<>:11: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_10583/3715806493.py:11: SyntaxWarning: invalid escape sequence '\.'
  verses = re.split('/|,|\.', stanza.text)
